# Cleaning Up

This notebook demonstrates how to clean up all the resources created in the previous set of notebooks.

In [1]:
import boto3
import json

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [3]:
%store -r

Please note these steps rely on each step being fully completed before the other will work. If you receive an error, wait a minute or two and try again.

## Clean up Campaigns

Start by cleaning up the campaigns. You need to delete campaigns before you can delete the solution versions upon which they are based. 

The code below will delete the Campaigns you created.

In [4]:
personalize.delete_campaign(
    campaignArn = userpersonalization_campaign_arn
)
personalize.delete_campaign(
    campaignArn = sims_campaign_arn
)
personalize.delete_campaign(
    campaignArn = rerank_campaign_arn
)

{'ResponseMetadata': {'RequestId': '9d1f7f2e-0b27-446e-b586-b7cb81c84672',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 18 Mar 2021 15:48:21 GMT',
   'x-amzn-requestid': '9d1f7f2e-0b27-446e-b586-b7cb81c84672',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up event trackers

Next, clean up any event trackers. The code below will delete your event tracker.

In [5]:
personalize.delete_event_tracker(
    eventTrackerArn = event_tracker_arn
)

{'ResponseMetadata': {'RequestId': 'd66fbbb9-3d54-475b-8b5e-356ac0d79f4d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 18 Mar 2021 15:48:24 GMT',
   'x-amzn-requestid': 'd66fbbb9-3d54-475b-8b5e-356ac0d79f4d',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up Filters

Next, clean up any filters. The code below iterate over all filters you created and delete them. Note, if you get an error about a campaign, that is fine, wait for 2 minutes then try the cell below again. 

Note: You will not be able to delete the filters until the campaign deletion completes, which may take a few minutes

In [8]:
personalize.delete_filter(filterArn = genre_filter_arn)

for filter_arn in interaction_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in decade_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )

ResourceInUseException: An error occurred (ResourceInUseException) when calling the DeleteFilter operation: The dataset group the filter belongs to is referenced by a campaign that's being actively created. Please wait for the resource creation to complete, and try the DeleteFilter call again

## Clean up solutions

Next, clean up the solutions. The code below will delete the solutions that were created.

In [ ]:
personalize.delete_solution(
    solutionArn = user_personalization_solution_arn
)
personalize.delete_solution(
    solutionArn = sims_solution_arn
)
personalize.delete_solution(
    solutionArn = rerank_solution_arn
)

## Clean up datasets

Next, clean up the datasets. The code below will delete the item and interactions datasets.

In [ ]:
personalize.delete_dataset(
    datasetArn = items_dataset_arn
)
personalize.delete_dataset(
    datasetArn = interactions_dataset_arn
)

## Clean up the schemas

Next, clean up the schemas. You create an interaction and item schema, delete them.

In [ ]:
personalize.delete_schema(
    schemaArn = interaction_schema_arn
)
personalize.delete_schema(
    schemaArn = itemmetadataschema_arn
)

## Clean up the dataset groups

Finally, clean up the dataset group:

In [ ]:
personalize.delete_dataset_group(
    datasetGroupArn = dataset_group_arn
)

## Clean up the S3 bucket and IAM role

Start by deleting the role, then empty the bucket, then delete the bucket.

In [ ]:
iam = boto3.client('iam')

Identify the name of the role you want to delete.

You cannot delete an IAM role which still has policies attached to it. So after you have identified the relevant role, let's list the attached policies of that role.

In [ ]:
iam.list_attached_role_policies(
    RoleName = role_name
)

You need to detach the policies in the result above using the code below. Repeat for each attached policy.

In [ ]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
)

In [ ]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

Finally, you should be able to delete the IAM role.

In [ ]:
iam.delete_role(
    RoleName = role_name
)

To delete an S3 bucket, it first needs to be empty. The easiest way to delete an S3 bucket, is just to navigate to S3 in the AWS console, delete the objects in the bucket, and then delete the S3 bucket itself.